# Analysis of Slave Trade Profits - November Coding Session

*Gerhard de Kok*

This Jupyter notebook is a result of a coding session on 15 November 2021. Main goals are:
1. To load the entirety of our data
2. To try some ROI calculations on this data

In [169]:
# First, import the Pandas library
import pandas as pd
import numpy as np

# Next, we import our database, which comes in several files. Each author has her/his own subset of data
ventures_gdk = pd.read_csv('../Venture database GK.csv', index_col=0)
cashflows_gdk = pd.read_csv('../Cash flow database GK.csv')
ventures_gd = pd.read_csv('../Venture database GD.csv', index_col=0)
cashflows_gd = pd.read_csv('../Cash flow database GD.csv')
ventures_kr = pd.read_csv('../Venture database KR - new.csv', index_col=0)
cashflows_kr = pd.read_csv('../Cash flow database KR - new.csv')
ventures_mr = pd.read_csv('../Venture database MR.csv', index_col=0)
cashflows_mr = pd.read_csv('../Cash flow database MR- new.csv')
ventures_dr = pd.read_csv('../Venture database DR.csv', index_col=0)
cashflows_dr = pd.read_csv('../Cash flow database DR.csv')

# CLEANING THE DATA
# 1) Guillaume's cashflows database includes a 'line number', which the others do not. We drop that here
cashflows_gd.drop('Line_number', axis=1, inplace=True)
# 2) Miguel's cashflows database has different headers
cashflows_mr.columns = cashflows_gdk.columns

# Now, we merge all of the above datasets into two dataframes: ventures and cashflows
ventures = pd.concat([ventures_gdk, ventures_gd, ventures_kr, ventures_dr])
cashflows = pd.concat([cashflows_gdk, cashflows_gd, cashflows_kr, cashflows_dr])

# Reset the cashflows index, becauses it didn't merge nicely
cashflows = cashflows.reset_index()
cashflows.drop('index', axis=1, inplace=True)

# We also import the Transatlantic Slave Trade Database (TSTD, slavevoyages.org, 
# 2020 release of Voyages database, in SPSS-format)
# The TSTD source file can be downloaded from https://slavevoyages.org/voyage/downloads
# Note: in order to read SPSS-files, the Python dependency pyreadstat needs to be installed
TSTD = pd.read_spss('tastdb-exp-2020.sav')

# The TSTD has a special format for dates. This function easily converts TSTD-dates into the
# flexible Python datetime format
# Thanks to Judith for discovering the TSTD stores dates in 'seconds from 1582-10-14'
import datetime
def normalizedate(tstddate):
    """""
    Function to normalize the dates from the TSTD
    """
    return (datetime.datetime(1582, 10, 14) + datetime.timedelta(seconds = tstddate))

# The following function can be called upon to pull data from the TSTD
def pull_from_tstd(tstd_id, tstd_variable, isdate=False):
    """"
    Function to pull data from the TSTD
    tstd_id = the Voyage ID of the TSTD
    tstd_variable = the name of the variable to lookup in the TSTD (can be found in TSTD Handbook)
    isdate = optionally set to True if fetching a date, which will then be converted to Python datetime
    If function is given an empty TSTD ID or the TSTD variable holds no data, 'NA' is returned
    """
    if pd.isnull(tstd_id):
        return pd.NA
    
    if not pd.isnull(TSTD[tstd_variable].loc[(TSTD['VOYAGEID'] == tstd_id)].iloc[0]):
        if isdate == False:
            return TSTD[tstd_variable].loc[(TSTD['VOYAGEID'] == tstd_id)].iloc[0]
        else:
            return normalizedate(TSTD[tstd_variable].loc[(TSTD['VOYAGEID'] == tstd_id)].iloc[0])
    else:
        return pd.NA

Now let's try and to some analysis

In [170]:
# Create an empty list to hold the data on the ROI of voyages
roi_list = []

# Now we will populate the list with the voyages
for voyage in cashflows['VentureID'].unique():      
    
    # BANDAID: DROP NR039
    try:
        test = ventures.loc[voyage, 'Type of venture']
    except:
        continue
        
    # At present, we will only concern ourselves with 'voyages', not 'ships'    
    if (ventures.loc[voyage, 'Type of venture'] != 'Voyage'):
        continue
    
    # Retrieve whether data on outlays and returns are complete
    # If data are complete, we can assume the figures in the cashflow-
    # database can be used to find the ROI of the voyage
    if not (ventures.loc[voyage, 'Complete data on outlays?'] == 'yes' and ventures.loc[voyage, 'Complete data on returns?'] == 'yes'):
        continue
           
    # Retrieve the name of the ship used for the voyage
    ship_name = ventures.loc[voyage, 'Name of the ship']
    
    # Retrieve the nationality
    nationality = ventures.loc[voyage, 'Nationality']
    
    # Calculate the investment, returns and costs on return
    investment = cashflows['value'].loc[(cashflows['VentureID'] == voyage) & (cashflows['timing'] == 'Outfitting') & (cashflows['typeofcashflow'] == 'Expenditure')].sum()
    returns = cashflows['value'].loc[(cashflows['VentureID'] == voyage) & (cashflows['timing'] == 'Return') & (cashflows['typeofcashflow'] == 'Return')].sum()
    costonreturn = cashflows['value'].loc[(cashflows['VentureID'] == voyage) & (cashflows['timing'] == 'Return') & (cashflows['typeofcashflow'] == 'Expenditure')].sum()
 
    # Use the above-mentioned variables to calculate the gross returns, result and the ROI
    gross_returns = returns - costonreturn
    result = gross_returns - investment
    if investment != 0:
        ROI = result/investment * 100
    else:
        ROI = 0
    
    # To check: let's add the value for 'Profits reported in source'
    source_profit = ventures.loc[voyage, 'Profits reported in source']
       
    # Lookup the venture's TSTD ID
    tstd_id = ventures.loc[voyage, 'Voyage-ID in TSTD']
    if not pd.isnull(tstd_id):
        try:
            tstd_id = int(tstd_id)
        except:
            tstd_id = None
    
    # Pull the date of return from the TSTD if necessary
    if not pd.isnull(ventures.loc[voyage, 'Date of return to port of outfitting']):
        date_return = datetime.datetime.strptime(ventures.loc[voyage, 'Date of return to port of outfitting'],"%Y-%m-%d")
    else:
        date_return = pull_from_tstd(tstd_id, 'DATEEND', isdate=True)
        
    # Let's get just the year
    if not pd.isnull(date_return):
        date_return = date_return.year
    
    # Pull the (primary) region of purchase of slaves from the TSTD if necessary
    if not pd.isnull(ventures.loc[voyage, 'Place of purchase']):
        regionpurchase = ventures.loc[voyage, 'Place of purchase']
    else:
        regionpurchase = pull_from_tstd(tstd_id, 'MAJBYIMP')

    # Pull the number of slaves bought in Africa from the TSTD if necessary
    if not pd.isnull(ventures.loc[voyage, 'Number of slaves purchased']):
        number_purchased = ventures.loc[voyage, 'Number of slaves purchased']
    else:
        number_purchased = pull_from_tstd(tstd_id, 'SLAXIMP')
    
    # Construct a list with data on this voyage
    roi_voyage = [voyage, ship_name, nationality, date_return, regionpurchase, number_purchased, investment, gross_returns, result, source_profit, cashflows['currency'].loc[(cashflows['VentureID'] ==  voyage)].iloc[0], ROI]
    
    # Append that list to the aforementioned list to create a list of lists
    roi_list.append(roi_voyage)
    
# Create a Pandas dataframe from the aforementioned list of lists
fullanalysis = pd.DataFrame.from_records(roi_list, columns=['VoyageID', 'Ship', 'Nationality', 'End of voyage', 'Region of slave purchase', 'Number of slaves', 'Investment', 'Gross returns', 'Result', 'Result in source', 'Currency', 'ROI'], index='VoyageID')

In [171]:
# Let's get some stats for our data
check_fullROI = fullanalysis[['Nationality', 'ROI']].dropna()
check_fullROI['ROI'].describe()

count    240.000000
mean       3.824419
std       59.523465
min     -405.200913
25%      -13.874384
50%        3.554484
75%       22.493267
max      545.424698
Name: ROI, dtype: float64

In [178]:
# Check overall differences in ROI per nationality
check_fullROI.groupby('Nationality').agg(['mean', 'count']).round(2)

ROI      
              mean count
Nationality             
Danish       37.06     5
Dutch         3.69    96
English      11.04   103
French      -21.09    36

In [179]:
filt = fullanalysis['Nationality'] == 'French'
fullanalysis[filt].head(n=80)

,Ship,Nationality,End of voyage,Region of slave purchase,Number of slaves,Investment,Gross returns,Result,Result in source,Currency,ROI
VoyageID,,,,,,,,,,,
GD002,Le Saint-René,French,1766,Adra,220,1690.229167,2.361217e+03,6.709875e+02,NaN,Livres tournois,39.698019
GD004,La Glaneuse,French,1773,"West Central Africa and St. Helena, port unspe...",172,104755.000000,0.000000e+00,-1.047550e+05,NaN,Livres tournois,-100.000000
GD005,Le Saint-Jean-Baptiste,French,1772,West Central Africa and St. Helena,571,240616.000000,0.000000e+00,-2.406160e+05,NaN,Livres tournois,-100.000000
GD006,La Jeune Reine,French,<NA>,Gorée,<NA>,100036.000000,0.000000e+00,-1.000360e+05,NaN,Livres tournois,-100.000000
GD017,Le Maréchal de Luxembourg,French,1769,Côte d'or,691,344014.000000,4.182324e+04,-3.021908e+05,NaN,Livres tournois,-87.842576
GD023,L'Union,French,<NA>,Sénégal,<NA>,16936.500000,0.000000e+00,-1.693650e+04,NaN,Livres tournois,-100.000000
GD024,La Madame,French,1789,Mozambic,586,426174.550000,7.427545e+05,3.165799e+05,NaN,Livres tournois,74.284102
GD027,Le Mars,French,1769,Côte d'or,373,158686.287500,6.905202e+04,-8.963427e+04,NaN,Livres tournois,-56.485202
GD061,La Jeanne-Thérèse,French,1786,Bight of Benin,300,227119.000000,1.214873e+05,-1.056317e+05,NaN,Livres tournois,-46.509412


In [173]:
filt = cashflows['VentureID'] == 'GD002'
cashflows[filt].head(n=20)

,VentureID,typeofcashflow,specification,value,meansofpaymentreturn,intermediarytradingoperation,estimate,dateoftransaction,timing,currency,remarks
1473,GD002,Expenditure,Outfiting (Armement),1690.229167,NaN,NaN,NaN,1764,Outfitting,Livres tournois,NaN
1474,GD002,Expenditure,Since outfitting,238.491667,NaN,NaN,NaN,NaN,After outfitting,Livres tournois,NaN
1475,GD002,Return,Not specified,979.991667,NaN,NaN,NaN,1766-08-22,Return,Livres tournois,NaN
1476,GD002,Return,Not specified,190.750000,NaN,NaN,NaN,1767-07-24,Return,Livres tournois,NaN
1477,GD002,Return,Not specified,95.733333,NaN,NaN,NaN,1767-07-24,Return,Livres tournois,NaN
1478,GD002,Return,Not specified,293.575000,NaN,NaN,NaN,1768-01-12,Return,Livres tournois,NaN
1479,GD002,Return,Not specified,176.658333,NaN,NaN,NaN,1768-01-12,Return,Livres tournois,NaN
1480,GD002,Return,Not specified,241.612500,NaN,NaN,NaN,1768-01-12,Return,Livres tournois,NaN
1481,GD002,Return,Not specified,91.820833,NaN,NaN,NaN,1768-09-13,Return,Livres tournois,NaN
1482,GD002,Return,Not specified,61.125000,NaN,NaN,NaN,1770-05-29,Return,Livres tournois,NaN


In [174]:
cashflows[filt].head(n=20).dtypes

VentureID                        object
typeofcashflow                   object
specification                    object
value                           float64
meansofpaymentreturn             object
intermediarytradingoperation    float64
estimate                         object
dateoftransaction                object
timing                           object
currency                         object
remarks                          object
dtype: object

In [175]:
cashflows[filt]['value']

1473    1690.229167
1474     238.491667
1475     979.991667
1476     190.750000
1477      95.733333
1478     293.575000
1479     176.658333
1480     241.612500
1481      91.820833
1482      61.125000
1483      39.958333
1484      41.100000
1485      64.641667
1486      84.250000
Name: value, dtype: float64

In [176]:
cashflows[filt]['value'].sum()

4289.937500000004